In [1]:
from pyabc.crystal.utils import non_dup_hnfs
from pyabc.io.vasp import read_vasp, write_vasp
from pyabc.crystal.structure import symbol2number as s2n
from pyabc.crystal.derive import ConfigurationGenerator, PermutationGroup

import os
import shutil
import numpy

In [2]:
def _get_sites(l_atoms, ele, l_sub):
    ele_n = s2n(ele)
    l_sub_n = [s2n(sub_n) for sub_n in l_sub]
    sites = []
    for a in l_atoms:
        if a == ele_n:
            sites.append(tuple([a] + l_sub_n))
        else:
            sites.append(tuple([a]))
    return sites

In [3]:
symprec = 1e-3
comprec = 1e-3

dimension = 2

comment = 'BxNxCy-FROM-BN'

In [4]:
cell = read_vasp("./BN-relax.vasp")
cg = ConfigurationGenerator(cell, symprec)

In [5]:
if not os.path.exists('./BNC-from-BN'):
    os.mkdir('./BNC-from-BN')
else:
    shutil.rmtree('./BNC-from-BN')
    os.mkdir('./BNC-from-BN')

In [6]:
idx = 0

# 在两重循环中分别替换B和N
# 此处产生四个C的所有结构，也就是分子式为：B14N14C4的所有结构
# e_num = (13, 3)

# 以BN 4x4平面作为初始构型

# 首先以C替换B
for i in range(1, 4):
    e_num = (16-i, i)
    element = 'B'
    substitutes = 'C'
    supercell = cell
    sites = _get_sites(supercell.atoms.tolist(), element, substitutes)
    cg = ConfigurationGenerator(supercell, symprec)
    for c, _ in cg.cons_specific_cell(sites, e_num=e_num, symprec=symprec):
        # 对于新得到的结构，使用C替换N
        supercell = c
        element = 'N'
        substitutes = 'C'
        sites = _get_sites(supercell.atoms.tolist(), element, substitutes)
        cg = ConfigurationGenerator(supercell, symprec)
        for c, _ in cg.cons_specific_cell(sites, e_num=e_num, symprec=symprec):
            filename = '{:s}_id{:d}'.format(comment, idx)
            idx += 1
            file = os.path.join('./BNC-from-BN', filename)
    #         print(c)
            write_vasp(c, file)